In [1]:
import anndata
import numpy as np
import sys
import torch
import scvelo as scv
sys.path.append('../../../../')
import velovae as vv

In [2]:
dataset = 'Erythroid'
#adata = scv.datasets.gastrulation_erythroid()
adata = anndata.read_h5ad(f'data/{dataset}_pp.h5ad')
#standardize the cluster key
adata.obs["clusters"] = adata.obs["celltype"].to_numpy()

In [ ]:
vv.preprocess(adata, n_gene=2000, min_shared_counts=20)

In [7]:
model_path_base = f'checkpoints/{dataset}_notime'
figure_path_base = f'figures/{dataset}_notime'
data_path = f'data/velovae/continuous/{dataset}_notime'
gene_plot = ['Smim1', 'Blvrb', 'Hba-x', 'Lmo2']

# Informative Time Prior

In [8]:
capture_time = adata.obs['stage'].to_numpy()
tprior = np.array([float(x[1:]) for x in capture_time])
adata.obs['tprior'] = tprior - tprior.min()

# Vanilla VAE

In [9]:
figure_path = f'{figure_path_base}/Vanilla'
model_path = f'{model_path_base}/Vanilla'

torch.manual_seed(2022)
np.random.seed(2022)

vanilla_vae = vv.VanillaVAE(adata, 
                            tmax=20, 
                            device='cuda:0',
                            init_method='tprior',
                            init_key='tprior',
                            tprior=None)

vanilla_vae.train(adata, config={}, plot=False, gene_plot=gene_plot, figure_path=figure_path)

vanilla_vae.save_model(model_path, 'encoder', 'decoder')
vanilla_vae.save_anndata(adata, 'vanilla', data_path, file_name=f'{dataset}.h5ad')

Initialization using prior time.
Estimating ODE parameters...


  0%|          | 0/1000 [00:00<?, ?it/s]

Estimating the variance...


  0%|          | 0/1000 [00:00<?, ?it/s]

Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/1000 [00:00<?, ?it/s]

Gaussian Prior.
------------------------- Train a Vanilla VAE -------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
Total Number of Iterations Per Epoch: 54, test iteration: 106
********* Early Stop Triggered at epoch 187. *********
*********              Finished. Total Time =   0 h :  1 m : 39 s             *********
Final: Train ELBO = 448.562,           Test ELBO = 446.890


# VeloVAE

In [10]:
figure_path = f'{figure_path_base}/VeloVAE'
model_path = f'{model_path_base}/VeloVAE'

torch.manual_seed(2022)
np.random.seed(2022)

vae = vv.VAE(adata, 
             tmax=20, 
             dim_z=5, 
             device='cuda:0',
             init_method='tprior',
             init_key='tprior',
             tprior=None,
             init_ton_zero=False)


config = {
    'early_stop':9, 
    'train_ton':False
}
vae.train(adata, config=config, plot=False, gene_plot=gene_plot, figure_path=figure_path)

vae.save_model(model_path, 'encoder', 'decoder')
vae.save_anndata(adata, 'velovae', data_path, file_name=f'{dataset}.h5ad')

Initialization using prior time.
Estimating ODE parameters...


  0%|          | 0/1000 [00:00<?, ?it/s]

Estimating the variance...


  0%|          | 0/1000 [00:00<?, ?it/s]

Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/1000 [00:00<?, ?it/s]

Gaussian Prior.
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 54, test iteration: 106
*********       Stage 1: Early Stop Triggered at epoch 670.       *********
*********                      Stage  2                       *********
Cell-wise KNN Estimation.


  0%|          | 0/9815 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.009
Average Set Size: 422
Finished. Actual Time:   0 h :  0 m :  9 s
*********       Stage 2: Early Stop Triggered at epoch 767.       *********
*********              Finished. Total Time =   0 h :  7 m : 28 s             *********
Final: Train ELBO = 530.080,           Test ELBO = 518.562


# FullVB

In [11]:
figure_path = f'{figure_path_base}/FullVB'
model_path = f'{model_path_base}/FullVB'

torch.manual_seed(2022)
np.random.seed(2022)

full_vb = vv.VAEFullVB(adata, 
                       tmax=20, 
                       dim_z=5, 
                       device='cuda:0',
                       init_method='tprior',
                       init_key='tprior',
                       tprior=None,
                       init_ton_zero=False)

config = {
    'early_stop':9, 
    'train_ton':False,
}
full_vb.train(adata, config=config, plot=False, gene_plot=gene_plot, figure_path=figure_path)

full_vb.save_model(model_path, 'encoder', 'decoder')
full_vb.save_anndata(adata, 'fullvb', data_path, file_name=f'{dataset}.h5ad')

Initialization using prior time.
Estimating ODE parameters...


  0%|          | 0/1000 [00:00<?, ?it/s]

Estimating the variance...


  0%|          | 0/1000 [00:00<?, ?it/s]

Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/1000 [00:00<?, ?it/s]

Gaussian Prior.
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 54, test iteration: 106
*********       Stage 1: Early Stop Triggered at epoch 744.       *********
*********                      Stage  2                       *********
Cell-wise KNN Estimation.


  0%|          | 0/9815 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.009
Average Set Size: 412
Finished. Actual Time:   0 h :  0 m :  9 s
*********       Stage 2: Early Stop Triggered at epoch 1204.       *********
*********              Finished. Total Time =   0 h : 14 m :  3 s             *********
Final: Train ELBO = 524.880,           Test ELBO = 502.392


# Branching ODE

In [12]:
figure_path = f'{figure_path_base}/BrODE'
model_path = f'{model_path_base}/BrODE'

torch.manual_seed(2022)
np.random.seed(2022)
brode = vv.BrODE(adata, 'clusters', 'fullvb_time', 'fullvb_z')

brode.print_weight()

brode.train(adata, 'fullvb_time', 'clusters', plot=False, gene_plot=gene_plot, figure_path=figure_path)

brode.save_model(model_path, 'brode')
brode.save_anndata(adata, 'brode', data_path, file_name=f'{dataset}.h5ad')

vv.plot_transition_graph(adata, save=f'{figure_path_base}/transition.png')

Graph Partition
Number of partitions:  1
Computing type-to-type transition probability
Obtaining the MST in each partition
Initialization using type-specific dynamical model.
Estimating ODE parameters...


  0%|          | 0/1000 [00:00<?, ?it/s]

Estimating the variance...


  0%|          | 0/1000 [00:00<?, ?it/s]

                     Blood progenitors 1  Blood progenitors 2  Erythroid1  Erythroid2  Erythroid3
Blood progenitors 1                  0.0                  1.0         0.0         0.0         0.0
Blood progenitors 2                  0.0                  1.0         0.0         0.0         0.0
Erythroid1                           0.0                  1.0         0.0         0.0         0.0
Erythroid2                           0.0                  0.0         1.0         0.0         0.0
Erythroid3                           0.0                  0.0         0.0         1.0         0.0
------------------------ Train a Branching ODE ------------------------
                     Blood progenitors 1  Blood progenitors 2  Erythroid1  Erythroid2  Erythroid3
Blood progenitors 1                  0.0                  1.0         0.0         0.0         0.0
Blood progenitors 2                  0.0                  1.0         0.0         0.0         0.0
Erythroid1                           0.0      

# Evaluation

In [ ]:
cluster_edges = [('Blood progenitors 1', 'Blood progenitors 2'),
                 ('Blood progenitors 2', 'Erythroid1'),
                 ('Erythroid1', 'Erythroid2'),
                 ('Erythroid2', 'Erythroid3')]
vv.post_analysis(adata,
                 'eval',
                 ['Vanilla VAE', 'VeloVAE', 'FullVB', 'BrODE'],
                 ['vanilla', 'velovae', 'fullvb', 'brode'],
                 compute_metrics=True,
                 genes=gene_plot,
                 grid_size=(1,4),
                 plot_type=['all'],
                 save_path=data_path,
                 cluster_edges=cluster_edges)